In [1]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

# Задаем seed для повторяемости результатов
numpy.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 25
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))


Using TensorFlow backend.
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/t

170500096/170498071 [==============================] - 30s 0us/step


Train on 45000 samples, validate on 5000 samples
Epoch 1/25
 - 85s - loss: 1.7550 - accuracy: 0.3550 - val_loss: 1.3610 - val_accuracy: 0.4976
Epoch 2/25
 - 85s - loss: 1.2975 - accuracy: 0.5319 - val_loss: 1.0684 - val_accuracy: 0.6198
Epoch 3/25
 - 100s - loss: 1.1294 - accuracy: 0.5979 - val_loss: 0.9424 - val_accuracy: 0.6638
Epoch 4/25
 - 89s - loss: 1.0129 - accuracy: 0.6392 - val_loss: 0.9169 - val_accuracy: 0.6780
Epoch 5/25
 - 95s - loss: 0.9283 - accuracy: 0.6749 - val_loss: 0.8040 - val_accuracy: 0.7248
Epoch 6/25
 - 92s - loss: 0.8657 - accuracy: 0.6937 - val_loss: 0.7982 - val_accuracy: 0.7200
Epoch 7/25
 - 91s - loss: 0.8275 - accuracy: 0.7100 - val_loss: 0.7466 - val_accuracy: 0.7460
Epoch 8/25
 - 99s - loss: 0.7854 - accuracy: 0.7254 - val_loss: 0.7383 - val_accuracy: 0.7500
Epoch 9/25
 - 89s - loss: 0.7462 - accuracy: 0.7381 - val_loss: 0.7527 - val_accuracy: 0.7438
Epoch 10/25
 - 97s - loss: 0.7255 

In [4]:
import json

In [5]:
model_json = model.to_json()

In [8]:
json_file = open("ourmodel.json", "w")
json_file.write(model_json)
json_file.close()

In [9]:
model.save_weights("ourmodel.h5")